In [1]:
import cv2
import numpy as np
import os
from skimage import measure
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def process_image(image_path, scale):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        return [], "文件读取失败"
    
    _, binary_image = cv2.threshold(image, 254, 1, cv2.THRESH_BINARY_INV)
    binary_image = 1-binary_image
    labels = measure.label(binary_image, connectivity=2)
    
    border_labels = np.unique(np.concatenate((
        labels[0, :], labels[-1, :], labels[:, 0], labels[:, -1]
    )))
    mask = np.isin(labels, border_labels, invert=True)
    filtered_labels = labels * mask
    
    regions = measure.regionprops(filtered_labels)
    areas = [region.area for region in regions if region.area >= 5]
    
    if not areas:
        return [], "无符合要求的连通区域"

    short_side_length = min(image.shape)
    nanometer_per_pixel = scale / short_side_length
    areas_nm2 = [area * (nanometer_per_pixel ** 2) for area in areas]
    
    return areas_nm2, None

def process_folder(folder_path, scale):
    result = {}
    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            areas_nm2, error = process_image(image_path, scale)
            if error:
                print(f"{filename}: {error}")
            else:
                result[filename] = areas_nm2
                txt_path = os.path.join(folder_path, f"{filename}.txt")
                with open(txt_path, 'w') as f:
                    for area in areas_nm2:
                        f.write(f"{area:.2f}\n")
    return result

def print_statistics(result):
    for filename, areas in result.items():
        if areas:
            avg_area = np.mean(areas)
            std_dev_area = np.std(areas)
            print(f"{filename}: 平均面积 = {avg_area:.2f} nm², 标准差 = {std_dev_area:.2f} nm²")
        else:
            print(f"{filename}: 无符合要求的连通区域")

In [8]:
afm_result = process_folder('calculate_picu_d/AFM', scale = 600)
random_result = process_folder('calculate_picu_d/random', scale = 1000)

print("AFM 文件夹结果:")
print_statistics(afm_result)

print("Random 文件夹结果:")
print_statistics(random_result)

AFM 文件夹结果:
0-init.png: 平均面积 = 1792.68 nm², 标准差 = 2178.89 nm²
1-eq-high.png: 平均面积 = 2543.05 nm², 标准差 = 4911.68 nm²
10-sh-low-2.png: 平均面积 = 1338.65 nm², 标准差 = 1865.52 nm²
2-eq-low.png: 平均面积 = 1789.66 nm², 标准差 = 2389.60 nm²
3-st-high-1.png: 平均面积 = 2016.25 nm², 标准差 = 3895.76 nm²
4-st-high-2.png: 平均面积 = 1233.70 nm², 标准差 = 1732.94 nm²
5-st-low-1.png: 平均面积 = 1458.07 nm², 标准差 = 2313.53 nm²
6-st-low-2.png: 平均面积 = 1736.27 nm², 标准差 = 2466.08 nm²
7-sh-high-1.png: 平均面积 = 1787.28 nm², 标准差 = 3433.58 nm²
8-sh-high-2.png: 平均面积 = 1828.39 nm², 标准差 = 2986.54 nm²
9-sh-low-1.png: 平均面积 = 1177.38 nm², 标准差 = 1829.51 nm²
Random 文件夹结果:
0-eq-high.png: 平均面积 = 1708.09 nm², 标准差 = 5211.42 nm²
1-sh-high-1.png: 平均面积 = 1509.63 nm², 标准差 = 4029.17 nm²
2-sh-high-2.png: 平均面积 = 882.32 nm², 标准差 = 2276.49 nm²
3-eq-low.png: 平均面积 = 736.75 nm², 标准差 = 2339.90 nm²
4-sh-low-1.png: 平均面积 = 720.86 nm², 标准差 = 2134.71 nm²
5-sh-low-2.png: 平均面积 = 651.10 nm², 标准差 = 1878.41 nm²
6-st-high-1.png: 平均面积 = 3150.99 nm², 标准差 = 11374.34 nm²
7-st-hig

In [4]:
import os
import cv2
import numpy as np
from skimage.morphology import skeletonize
from skimage import img_as_bool
from scipy.ndimage import distance_transform_edt

In [ ]:
def calculate_num_of_bundle(image):
    binary_image = cv2.threshold(image, 254, 255, cv2.THRESH_BINARY_INV)[1] // 255
    
    dist_transform = distance_transform_edt(binary_image)
    
    skeleton = skeletonize(binary_image)
    
    skeleton_distances = dist_transform * skeleton
    
    sum_distances = skeleton_distances.sum()
    
    num_skeleton_pixels = skeleton.sum()
    
    num_of_bundle = sum_distances / num_skeleton_pixels if num_skeleton_pixels > 0 else 0
    
    return num_of_bundle

def resize_image(image, target_size):
    h, w = image.shape
    scale = target_size / min(h, w)
    new_h, new_w = int(h * scale), int(w * scale)
    resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_AREA)
    return resized_image

def process_images(folder_path):
    for subfolder in ['AFM', 'random']:
        path = os.path.join(folder_path, subfolder)
        for file_name in os.listdir(path):
            if file_name.endswith('.png'):
                file_path = os.path.join(path, file_name)
                image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                if subfolder == 'AFM': 
                    image = resize_image(image, 300) 
                elif subfolder == 'random': 
                    image = resize_image(image, 500)
                num_of_bundle = calculate_num_of_bundle(image)

                num_of_bundle = num_of_bundle / 1.715
                num_of_bundle = num_of_bundle * 2 - 1
                print(f"Image: {file_name}, num_of_bundle: {(num_of_bundle):.2f}")

In [6]:
folder_path = 'calculate_picu_d'
process_images(folder_path)

Image: 0-init.png, num_of_bundle: 1.51
Image: 1-eq-high.png, num_of_bundle: 1.58
Image: 10-sh-low-2.png, num_of_bundle: 1.70
Image: 2-eq-low.png, num_of_bundle: 1.63
Image: 3-st-high-1.png, num_of_bundle: 1.75
Image: 4-st-high-2.png, num_of_bundle: 1.76
Image: 5-st-low-1.png, num_of_bundle: 1.73
Image: 6-st-low-2.png, num_of_bundle: 1.68
Image: 7-sh-high-1.png, num_of_bundle: 1.81
Image: 8-sh-high-2.png, num_of_bundle: 1.78
Image: 9-sh-low-1.png, num_of_bundle: 1.72
Image: 0-eq-high.png, num_of_bundle: 1.74
Image: 1-sh-high-1.png, num_of_bundle: 1.99
Image: 2-sh-high-2.png, num_of_bundle: 2.53
Image: 3-eq-low.png, num_of_bundle: 1.70
Image: 4-sh-low-1.png, num_of_bundle: 1.70
Image: 5-sh-low-2.png, num_of_bundle: 1.71
Image: 6-st-high-1.png, num_of_bundle: 2.64
Image: 7-st-high-2.png, num_of_bundle: 14.57
